In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from cartopy import crs as ccrs, feature as cfeature
import datetime
from matplotlib.animation import FuncAnimation
plt.rcParams["animation.html"] = "jshtml" #for jupyterlab display
import glob
import geoviews as gv
import hvplot.xarray
import holoviews as hv
import bokeh
import numpy as np

In [ ]:
# import cartopy
# print(cartopy.__version__)

In [ ]:
def get_single_match(pattern):
    matches = glob.glob(pattern)
    if len(matches) == 1:
        return matches[0]
    elif len(matches) == 0:
        raise ValueError(f"No matches found")
    else:
        raise ValueError(f"Multiple matches found: {matches}")

## Open 1 Month: 09 2024 

In [ ]:
# %cd FLUXNET-Model-comparison/notebooks/

In [ ]:
filepath = '../../micasa-data/3hrly/2024/09/MiCASA_v1_flux_x3600_y1800_3hrly_202409*.nc4'
ds = xr.open_mfdataset(filepath, combine="by_coords", chunks={})['NEE']
ds

## Open 1 day (1 month daily avg looks all the same)

In [ ]:
filepath = '../../micasa-data/3hrly/2015/01/MiCASA_v1_flux_x3600_y1800_3hrly_20150101.nc4'
ds = xr.open_dataset(filepath)
ds

## Preprocess

In [ ]:
# Plot only North America, drop unused lat/lon
min_lon, max_lon = -170, -30
min_lat, max_lat = 10, 70
proj=ccrs.PlateCarree()

In [ ]:
ds_subset = ds.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon,max_lon))

# mask zeroes
ds_subset = ds_subset.where(ds_subset != 0)

In [ ]:
ds_plot = ds_subset['NEE']
ds_plot

# Hvplot, which can run via bokeh or matplotlib in the backend:

In [ ]:
from bokeh.models import PrintfTickFormatter
formatter = PrintfTickFormatter(format="%0e")
plot_opts = dict(colorbar=True, colorbar_opts={'formatter': formatter})

## Bokeh backend

### Widget slider won't change??

In [ ]:
hv.extension('bokeh') 
plot = ds_plot.hvplot.image(x="lon", y="lat", 
                                   groupby='time',
                       cmap='RdBu_r', colorbar=True,
                       clim=(-4e-7,4e-7),
                            geo=True, 
                            # projection=proj,
                            crs = proj,
                            # rasterize=True, 
                        # tiles='EsriImagery',
                      clabel='NEE (kg m-2 s-1)')

plot.opts(colorbar_opts={'formatter': formatter})
plot

### Still doesn't work:

In [ ]:
vmin = -max(abs(ds_plot.min().values),abs(ds_plot.max().values))
vmax = max(abs(ds_plot.min().values),abs(ds_plot.max().values))

ds_plot.hvplot(cmap='seismic',groupby='time', clim=(vmin,vmax),
             # geo=True, 
            projection=proj,
            xlim=(min_lon, max_lon), 
            ylim=(min_lat, max_lat),
            widget_type="scrubber", 
            framewise=True,
            widget_location="bottom",
                clabel='NEE (kg m-2 s-1)')

# Matplotlib backend

In [ ]:
# Try mpl backend
hv.extension('matplotlib')
plot = ds_plot.hvplot.image(x="lon", y="lat", 
                       cmap='RdBu_r', colorbar=True,
                       clim=(-4e-7,4e-7),
                            geo=True, 
                            rasterize=True, 
                      clabel='NEE (kg m-2 s-1)')

In [ ]:
plot

### Same issue as above

In [ ]:
# hv.help(hv.Image)

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6),subplot_kw= {'projection': ccrs.PlateCarree()});
# ax.set_extent([min_lon, max_lon,min_lat, max_lat], crs=proj);
# # ax.coastlines()
# # ax.add_feature(cfeature.LAND)
# ax.add_feature(cfeature.OCEAN)

# OLD Matplotlib animation (one day 01/01/2015)

In [ ]:
# Animation func
def animate(frame):
    ax.clear()
    ax.set_extent([-170,-30,-60,90], crs=ccrs.PlateCarree())
    
    plot = ds.isel(time=frame).plot.pcolormesh(
        ax=ax,
        add_colorbar=False,
        )
    title = str(ds.isel(time=frame)['time'].dt.strftime('%b %d %Y %H:%M').values)
    ax.set_title(title,pad=0)
    return plot

In [ ]:
filepath = '../../micasa-data/3hrly/2015/01/MiCASA_v1_flux_x3600_y1800_3hrly_20150101.nc4'
ds = xr.open_dataset(filepath)
ds

In [ ]:
ds = ds['NEE']

In [ ]:
# Initial plot
plt.ioff()
fig, ax = plt.subplots(figsize=(8,6),subplot_kw= {'projection': ccrs.PlateCarree()});
ax.set_extent([-170,-30,-60,90], crs=ccrs.PlateCarree());
initial_plot = ds.isel(time=0).plot.pcolormesh(ax=ax);
title = str(ds.isel(time=0)['time'].dt.strftime('%b %d %Y').values);
ax.set_title(title,pad=-0.1);

# Run animation
anim = FuncAnimation(fig=fig, func=animate, frames=len(ds.time))
anim

In [ ]:
# ax.set_extent([min_lon, max_lon,min_lat, max_lat], crs=proj);
# ax.coastlines()
# ax.add_feature(cfeature.LAND)
# ax.add_feature(cfeature.OCEAN)

In [ ]:
# Save output
# anim.save(filename="micasa_testanimation.gif", writer="pillow")